## Examples

In this notebook some examples are generated. Given a query, the model returns similar sentences

In [1]:
from collections import defaultdict, Counter
import sys
import os
import seaborn as sn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
from pathlib import Path
from pprint import pprint
import random 

import torch
from torch import nn, Tensor
from torch.utils import data as datautil
from torch.utils.data import Sampler, BatchSampler, DataLoader
import torch.nn.functional as F
from torch.optim import Optimizer
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence
from dacite import from_dict

from matplotlib.pyplot import figure
from scipy import spatial

sys.path.append('../') 
from train import mmModule, multimodal_loss, Cfg
from utils import read_metadata_subset, find_paths, relative_file_path, randomize_model
from dataloader import MMloader
from evaluate_topic import Evaluator

from omegaconf import OmegaConf
conf = OmegaConf.load("../config.yaml")

### Relocate some configuration files

In [2]:
# Rename configurations used, since they are one folder up.
conf.dataset_path = os.path.join("../", conf.dataset_path)
conf.sp_path = os.path.join("../", conf.sp_path)
conf.yamnet_query_embed_path = os.path.join("../", conf.yamnet_query_embed_path)
conf.yamnet_descr_embed_path = os.path.join("../", conf.yamnet_descr_embed_path)

conf.sent_topic_query_embed_dir = os.path.join("../", conf.sent_topic_query_embed_dir)
conf.sent_topic_descr_embed_dir = os.path.join("../", conf.sent_topic_descr_embed_dir)

metadata_testset, topics_df, topics_df_targets = read_metadata_subset(conf, traintest='test')

qs =  topics_df['query'].tolist()
ds =  topics_df['description'].tolist()

[utils] metadata loaded  105360
[utils] topics_df_targets loaded for test set 9426
[utils] metadata_subset loaded  9426


In [3]:
model_weights_path = '../logs/25m-gru_2022-07-06_07-25-51/output/full_model_weights.pt'

In [4]:
# Reading the metadata.
metadata_testset, topics_df, topics_df_targets = read_metadata_subset(conf, traintest='test')


[utils] metadata loaded  105360
[utils] topics_df_targets loaded for test set 9426
[utils] metadata_subset loaded  9426


In [5]:
# Rename model paths. 
model_path = Path(model_weights_path).parents[1]
model_config_path = os.path.join(model_path, 'config.json')

# Opening JSON file
f = open(model_config_path)
model_config = json.load(f)
CFG = from_dict(data_class=Cfg, data=model_config)
CFG.device = "cuda" if torch.cuda.is_available() else "cpu"
CFG.sp_path = conf.sp_path

# Create dataloader of test set.
data_loader = MMloader(CFG)

# Load the model.
full_model = mmModule(CFG)
full_model.load_state_dict(torch.load(model_weights_path,  map_location=CFG.device)) 
full_model = full_model.to(CFG.device)     
full_model.eval()

# Calculate embeddings for test set.
evaluator = Evaluator(CFG, model_path, full_model, data_loader, False)
max_samples = evaluator.get_max_data()
print(max_samples)

[MMloader] init from directory  ../../data/yamnet/processed/ train
[MMloader] train dataset loaded, length:  4301835
[MMloader] init from directory  ../../data/yamnet/processed/ val
[MMloader] val dataset loaded, length:  252398
[MMloader] init from directory  ../../data/yamnet/processed/ test
[MMloader] test dataset loaded, length:  1530873
1530752


In [7]:

# For now we create a pool of 128*20000 samples
max_samples = 128*10000
evaluator.encode_testset(max_samples) 

Calculating:  0 20000
Calculating:  1 20000
Calculating:  2 20000
Calculating:  3 20000
Calculating:  4 20000
Calculating:  5 20000
Calculating:  6 20000
Calculating:  7 20000
Calculating:  8 20000
Calculating:  9 20000
Calculating:  10 20000
Calculating:  11 20000
Calculating:  12 20000
Calculating:  13 20000
Calculating:  14 20000
Calculating:  15 20000
Calculating:  16 20000
Calculating:  17 20000
Calculating:  18 20000
Calculating:  19 20000
Calculating:  20 20000
Calculating:  21 20000
Calculating:  22 20000
Calculating:  23 20000
Calculating:  24 20000
Calculating:  25 20000
Calculating:  26 20000
Calculating:  27 20000
Calculating:  28 20000
Calculating:  29 20000
Calculating:  30 20000
Calculating:  31 20000
Calculating:  32 20000
Calculating:  33 20000
Calculating:  34 20000
Calculating:  35 20000
Calculating:  36 20000
Calculating:  37 20000
Calculating:  38 20000
Calculating:  39 20000
Calculating:  40 20000
Calculating:  41 20000
Calculating:  42 20000
Calculating:  43 2000

Calculating:  637 20000
Calculating:  638 20000
Calculating:  639 20000
Calculating:  640 20000
Calculating:  641 20000
Calculating:  642 20000
Calculating:  643 20000
Calculating:  644 20000
Calculating:  645 20000
Calculating:  646 20000
Calculating:  647 20000
Calculating:  648 20000
Calculating:  649 20000
Calculating:  650 20000
Calculating:  651 20000
Calculating:  652 20000
Calculating:  653 20000
Calculating:  654 20000
Calculating:  655 20000
Calculating:  656 20000
Calculating:  657 20000
Calculating:  658 20000
Calculating:  659 20000
Calculating:  660 20000
Calculating:  661 20000
Calculating:  662 20000
Calculating:  663 20000
Calculating:  664 20000
Calculating:  665 20000
Calculating:  666 20000
Calculating:  667 20000
Calculating:  668 20000
Calculating:  669 20000
Calculating:  670 20000
Calculating:  671 20000
Calculating:  672 20000
Calculating:  673 20000
Calculating:  674 20000
Calculating:  675 20000
Calculating:  676 20000
Calculating:  677 20000
Calculating:  67

Calculating:  1137 20000
Calculating:  1138 20000
Calculating:  1139 20000
Calculating:  1140 20000
Calculating:  1141 20000
Calculating:  1142 20000
Calculating:  1143 20000
Calculating:  1144 20000
Calculating:  1145 20000
Calculating:  1146 20000
Calculating:  1147 20000
Calculating:  1148 20000
Calculating:  1149 20000
Calculating:  1150 20000
Calculating:  1151 20000
Calculating:  1152 20000
Calculating:  1153 20000
Calculating:  1154 20000
Calculating:  1155 20000
Calculating:  1156 20000
Calculating:  1157 20000
Calculating:  1158 20000
Calculating:  1159 20000
Calculating:  1160 20000
Calculating:  1161 20000
Calculating:  1162 20000
Calculating:  1163 20000
Calculating:  1164 20000
Calculating:  1165 20000
Calculating:  1166 20000
Calculating:  1167 20000
Calculating:  1168 20000
Calculating:  1169 20000
Calculating:  1170 20000
Calculating:  1171 20000
Calculating:  1172 20000
Calculating:  1173 20000
Calculating:  1174 20000
Calculating:  1175 20000
Calculating:  1176 20000


Calculating:  1637 20000
Calculating:  1638 20000
Calculating:  1639 20000
Calculating:  1640 20000
Calculating:  1641 20000
Calculating:  1642 20000
Calculating:  1643 20000
Calculating:  1644 20000
Calculating:  1645 20000
Calculating:  1646 20000
Calculating:  1647 20000
Calculating:  1648 20000
Calculating:  1649 20000
Calculating:  1650 20000
Calculating:  1651 20000
Calculating:  1652 20000
Calculating:  1653 20000
Calculating:  1654 20000
Calculating:  1655 20000
Calculating:  1656 20000
Calculating:  1657 20000
Calculating:  1658 20000
Calculating:  1659 20000
Calculating:  1660 20000
Calculating:  1661 20000
Calculating:  1662 20000
Calculating:  1663 20000
Calculating:  1664 20000
Calculating:  1665 20000
Calculating:  1666 20000
Calculating:  1667 20000
Calculating:  1668 20000
Calculating:  1669 20000
Calculating:  1670 20000
Calculating:  1671 20000
Calculating:  1672 20000
Calculating:  1673 20000
Calculating:  1674 20000
Calculating:  1675 20000
Calculating:  1676 20000


Calculating:  2137 20000
Calculating:  2138 20000
Calculating:  2139 20000
Calculating:  2140 20000
Calculating:  2141 20000
Calculating:  2142 20000
Calculating:  2143 20000
Calculating:  2144 20000
Calculating:  2145 20000
Calculating:  2146 20000
Calculating:  2147 20000
Calculating:  2148 20000
Calculating:  2149 20000
Calculating:  2150 20000
Calculating:  2151 20000
Calculating:  2152 20000
Calculating:  2153 20000
Calculating:  2154 20000
Calculating:  2155 20000
Calculating:  2156 20000
Calculating:  2157 20000
Calculating:  2158 20000
Calculating:  2159 20000
Calculating:  2160 20000
Calculating:  2161 20000
Calculating:  2162 20000
Calculating:  2163 20000
Calculating:  2164 20000
Calculating:  2165 20000
Calculating:  2166 20000
Calculating:  2167 20000
Calculating:  2168 20000
Calculating:  2169 20000
Calculating:  2170 20000
Calculating:  2171 20000
Calculating:  2172 20000
Calculating:  2173 20000
Calculating:  2174 20000
Calculating:  2175 20000
Calculating:  2176 20000


...

Calculating:  17662 20000
Calculating:  17663 20000
Calculating:  17664 20000
Calculating:  17665 20000
Calculating:  17666 20000
Calculating:  17667 20000
Calculating:  17668 20000
Calculating:  17669 20000
Calculating:  17670 20000
Calculating:  17671 20000
Calculating:  17672 20000
Calculating:  17673 20000
Calculating:  17674 20000
Calculating:  17675 20000
Calculating:  17676 20000
Calculating:  17677 20000
Calculating:  17678 20000
Calculating:  17679 20000
Calculating:  17680 20000
Calculating:  17681 20000
Calculating:  17682 20000
Calculating:  17683 20000
Calculating:  17684 20000
Calculating:  17685 20000
Calculating:  17686 20000
Calculating:  17687 20000
Calculating:  17688 20000
Calculating:  17689 20000
Calculating:  17690 20000
Calculating:  17691 20000
Calculating:  17692 20000
Calculating:  17693 20000
Calculating:  17694 20000
Calculating:  17695 20000
Calculating:  17696 20000
Calculating:  17697 20000
Calculating:  17698 20000
Calculating:  17699 20000
Calculating:

Calculating:  18172 20000
Calculating:  18173 20000
Calculating:  18174 20000
Calculating:  18175 20000
Calculating:  18176 20000
Calculating:  18177 20000
Calculating:  18178 20000
Calculating:  18179 20000
Calculating:  18180 20000
Calculating:  18181 20000
Calculating:  18182 20000
Calculating:  18183 20000
Calculating:  18184 20000
Calculating:  18185 20000
Calculating:  18186 20000
Calculating:  18187 20000
Calculating:  18188 20000
Calculating:  18189 20000
Calculating:  18190 20000
Calculating:  18191 20000
Calculating:  18192 20000
Calculating:  18193 20000
Calculating:  18194 20000
Calculating:  18195 20000
Calculating:  18196 20000
Calculating:  18197 20000
Calculating:  18198 20000
Calculating:  18199 20000
Calculating:  18200 20000
Calculating:  18201 20000
Calculating:  18202 20000
Calculating:  18203 20000
Calculating:  18204 20000
Calculating:  18205 20000
Calculating:  18206 20000
Calculating:  18207 20000
Calculating:  18208 20000
Calculating:  18209 20000
Calculating:

Calculating:  18631 20000
Calculating:  18632 20000
Calculating:  18633 20000
Calculating:  18634 20000
Calculating:  18635 20000
Calculating:  18636 20000
Calculating:  18637 20000
Calculating:  18638 20000
Calculating:  18639 20000
Calculating:  18640 20000
Calculating:  18641 20000
Calculating:  18642 20000
Calculating:  18643 20000
Calculating:  18644 20000
Calculating:  18645 20000
Calculating:  18646 20000
Calculating:  18647 20000
Calculating:  18648 20000
Calculating:  18649 20000
Calculating:  18650 20000
Calculating:  18651 20000
Calculating:  18652 20000
Calculating:  18653 20000
Calculating:  18654 20000
Calculating:  18655 20000
Calculating:  18656 20000
Calculating:  18657 20000
Calculating:  18658 20000
Calculating:  18659 20000
Calculating:  18660 20000
Calculating:  18661 20000
Calculating:  18662 20000
Calculating:  18663 20000
Calculating:  18664 20000
Calculating:  18665 20000
Calculating:  18666 20000
Calculating:  18667 20000
Calculating:  18668 20000
Calculating:

Calculating:  19131 20000
Calculating:  19132 20000
Calculating:  19133 20000
Calculating:  19134 20000
Calculating:  19135 20000
Calculating:  19136 20000
Calculating:  19137 20000
Calculating:  19138 20000
Calculating:  19139 20000
Calculating:  19140 20000
Calculating:  19141 20000
Calculating:  19142 20000
Calculating:  19143 20000
Calculating:  19144 20000
Calculating:  19145 20000
Calculating:  19146 20000
Calculating:  19147 20000
Calculating:  19148 20000
Calculating:  19149 20000
Calculating:  19150 20000
Calculating:  19151 20000
Calculating:  19152 20000
Calculating:  19153 20000
Calculating:  19154 20000
Calculating:  19155 20000
Calculating:  19156 20000
Calculating:  19157 20000
Calculating:  19158 20000
Calculating:  19159 20000
Calculating:  19160 20000
Calculating:  19161 20000
Calculating:  19162 20000
Calculating:  19163 20000
Calculating:  19164 20000
Calculating:  19165 20000
Calculating:  19166 20000
Calculating:  19167 20000
Calculating:  19168 20000
Calculating:

Calculating:  19596 20000
Calculating:  19597 20000
Calculating:  19598 20000
Calculating:  19599 20000
Calculating:  19600 20000
Calculating:  19601 20000
Calculating:  19602 20000
Calculating:  19603 20000
Calculating:  19604 20000
Calculating:  19605 20000
Calculating:  19606 20000
Calculating:  19607 20000
Calculating:  19608 20000
Calculating:  19609 20000
Calculating:  19610 20000
Calculating:  19611 20000
Calculating:  19612 20000
Calculating:  19613 20000
Calculating:  19614 20000
Calculating:  19615 20000
Calculating:  19616 20000
Calculating:  19617 20000
Calculating:  19618 20000
Calculating:  19619 20000
Calculating:  19620 20000
Calculating:  19621 20000
Calculating:  19622 20000
Calculating:  19623 20000
Calculating:  19624 20000
Calculating:  19625 20000
Calculating:  19626 20000
Calculating:  19627 20000
Calculating:  19628 20000
Calculating:  19629 20000
Calculating:  19630 20000
Calculating:  19631 20000
Calculating:  19632 20000
Calculating:  19633 20000
Calculating:

## Return top-k samples from a query

In [8]:

def get_topk_fromquery(evaluator, query, k=5):
    print("Input sentence: ", query)
    sent_txt_embed = evaluator.text_to_embed(query)
    
    # Create similarity matrix
    sim = (100.0 * sent_txt_embed @ evaluator.text_encoding.T)
    
    # return topK
    sim = sim.softmax(dim=-1)
    values, indices = sim.topk(k)
    topk_similar = [evaluator.all_sents[i] for i in indices]
    return topk_similar[0]


In [273]:
query = "Podcast about dogs."
topk_similar = get_topk_fromquery(evaluator, query, k=5)
for sent in topk_similar:
    print("\t", sent)

Input sentence:  Podcast about dogs.
	 I'm going to talk about for dogs.
	 I'm obsessed with dogs.
	 It just says dogs.
	  Because I love dogs.
	 Thank God we both have dogs.


In [9]:

query = 'People are starving.'
topk_similar = get_topk_fromquery(evaluator, query, k=5)
for sent in topk_similar:
    print("\t", sent)

Input sentence:  People are starving.
	 We are starving.
	 People are still starving.
	 You're not starving.
	 I was starving.
	 It's starving.


In [10]:
query = 'That is nice.'
topk_similar = get_topk_fromquery(evaluator, query, k=5)
for sent in topk_similar:
    print("\t", sent)

Input sentence:  That is nice.
	 That is nice.
	  I was nice.
	 That was nice.
	 That was nice.
	 That was nice.


In [11]:
query = 'Dr. Gerber.'
topk_similar = get_topk_fromquery(evaluator, query, k=5)
for sent in topk_similar:
    print("\t", sent)


Input sentence:  Dr. Gerber.
	 Dr. Gerber.
	 He'd been a carpenter.
	 Dr. Pepper.
	 Dr. Bacher.
	  It buffer.com.


In [271]:

query = "Podcast about therapy."
topk_similar = get_topk_fromquery(evaluator, query, k=5)
for sent in topk_similar:
    print("\t", sent)

	 It was therapy.
	 I was just going to come a therapist.
	 Alternative therapy.
	 Welcome to Thrift therapy.
	 Physical therapy.


In [278]:

query = "Podcast about self-help."
topk_similar = get_topk_fromquery(evaluator, query, k=5)
for sent in topk_similar:
    print("\t", sent)


Input sentence:  Podcast about self-help books.
	 I find those self-help books.
	 We have enough good self-help stuff.
	 We were talking about self-doubt.
	 Let's go out self-love.
	 I find those self-help books.


### Return top-k from a query (cross modal)

In [23]:

def get_topk(evaluator, idx, mod='text-text', k=5):
    sent = evaluator.all_sents[idx]
    print("Input sentence: ", sent)
    
    # Retreive embeddings
    sent_txt_embed = evaluator.text_encoding[idx]
    sent_audio_embed = evaluator.audio_encoding[idx]
    
    # Create similarity matrices
    if mod=='text-text':
        sim = (100.0 * sent_txt_embed @ evaluator.text_encoding.T)
    elif mod=='text-audio':
        sim = (100.0 * sent_txt_embed @ evaluator.audio_encoding.T)
    elif mod=='audio-text':
        sim = (100.0 * sent_audio_embed @ evaluator.text_encoding.T)
    elif mod=='audio-audio':
        sim = (100.0 * sent_audio_embed @ evaluator.audio_encoding.T)
    
    # return topK
    sim = sim.softmax(dim=-1)
    values, indices = sim.topk(k)
    topk_similar = [evaluator.all_sents[i] for i in indices]
    return topk_similar


modalities = ['text-text', 'text-audio', 'audio-text', 'audio-audio']

In [63]:

idx = random.randint(0, len(evaluator.all_sents))

for mod in modalities:
    print("\nMost similar for id {} and modality {}".format(idx, mod))
    topk_similar = get_topk(evaluator, idx, mod=mod, k=5)
    for sent in topk_similar:
        print("prediction: \t", sent) 
        


Most similar for id 178989 and modality text-text
Input sentence:  It's so strange.
prediction: 	 It's so strange.
prediction: 	 It's so strange.
prediction: 	 It was so strange.
prediction: 	 That's so strange.
prediction: 	 That's so strange.

Most similar for id 178989 and modality text-audio
Input sentence:  It's so strange.
prediction: 	 It's so strange.
prediction: 	 That's so strange.
prediction: 	 He so strange.
prediction: 	 This is so strange.
prediction: 	 That's so strange.

Most similar for id 178989 and modality audio-text
Input sentence:  It's so strange.
prediction: 	 It's so strange.
prediction: 	 Strange.
prediction: 	 It was so strange.
prediction: 	 It's so strange.
prediction: 	 It's so strange.

Most similar for id 178989 and modality audio-audio
Input sentence:  It's so strange.
prediction: 	 It's so strange.
prediction: 	 It's so strange.
prediction: 	 That was so strange.
prediction: 	 He goes.
prediction: 	 Yes, this is strange.


In [174]:

idx = random.randint(0, len(evaluator.all_sents))

for mod in modalities:
    print("\nMost similar for id {} and modality {}".format(idx, mod))
    topk_similar = get_topk(evaluator, idx, mod=mod, k=5)
    for sent in topk_similar:
        print("prediction: \t", sent)
        


Most similar for id 863328 and modality text-text
Input sentence:  What are you most thankful for this Thanksgiving?
prediction: 	 What are you most thankful for this Thanksgiving?
prediction: 	 What do you most thankful for for Thanksgiving?
prediction: 	 What are you plans for this wonderful Thanksgiving.
prediction: 	 I'm also grateful for sanity.
prediction: 	 Also intimacy.

Most similar for id 863328 and modality text-audio
Input sentence:  What are you most thankful for this Thanksgiving?
prediction: 	 Hey, what are you most thankful for this Thanksgiving.
prediction: 	 I was think of Thanksgiving.
prediction: 	 Tell them what you're thankful for.
prediction: 	 Happy Thanksgiving again.
prediction: 	 What are you most thankful for this Thanksgiving?

Most similar for id 863328 and modality audio-text
Input sentence:  What are you most thankful for this Thanksgiving?
prediction: 	 Let's go hold some heart space together.
prediction: 	 What are you most thankful for this Thanksgi

In [288]:

idx = random.randint(0, len(evaluator.all_sents))

for mod in modalities:
    print("\nMost similar for id {} and modality {}".format(idx, mod))
    topk_similar = get_topk(evaluator, idx, mod=mod, k=5)
    for sent in topk_similar:
        print("prediction: \t", sent)
        


Most similar for id 846526 and modality text-text
Input sentence:  I have a weight-loss goal.
prediction: 	 I have a weight-loss goal.
prediction: 	 Those are my goals.
prediction: 	 It makes or sport.
prediction: 	 I lost a ton of weight.
prediction: 	 So that was the score score score.

Most similar for id 846526 and modality text-audio
Input sentence:  I have a weight-loss goal.
prediction: 	 That's weight loss.
prediction: 	 Yeah, what are your next goals?
prediction: 	 I lost a ton of weight.
prediction: 	 I have a weight-loss goal.
prediction: 	 Those are my goals

Most similar for id 846526 and modality audio-text
Input sentence:  I have a weight-loss goal.
prediction: 	 Yeah, what are your next goals?
prediction: 	 What are my personal goals? 
prediction: 	 Let's talk about weight.
prediction: 	 What about your muscle mass?
prediction: 	 I have a weight-loss goal

Most similar for id 846526 and modality audio-audio
Input sentence:  I have a weight-loss goal.
prediction: 	 I ha

In [253]:
idx = random.randint(0, len(evaluator.all_sents))

for mod in modalities:
    print("\nMost similar for id {} and modality {}".format(idx, mod))
    topk_similar = get_topk(evaluator, idx, mod=mod, k=5)
    for sent in topk_similar:
        print("prediction: \t", sent)
        


Most similar for id 178989 and modality text-text
Input sentence:  So I was depressed probably for about five years.
prediction: 	 So I was depressed probably for about five years.
prediction: 	 I was the one that was fucking depressed.
prediction: 	 My son has been drug-free now over five years.
prediction: 	 But I was desperate.
prediction: 	 stop struggling.

Most similar for id 178989 and modality text-audio
Input sentence:  So I was depressed probably for about five years.
prediction: 	 So I was depressed probably for about five years.
prediction: 	 So I practice for five years.
prediction: 	 They're probably psyched as hell.
prediction: 	 But I was desperate.
prediction: 	 I'm a very independent person.

Most similar for id 178989 and modality audio-text
Input sentence:  So I was depressed probably for about five years.
prediction: 	 So I was depressed probably for about five years.
prediction: 	 I'm like no I never said I was depressed.
prediction: 	 I had feelings of depressio

In [254]:
idx = random.randint(0, len(evaluator.all_sents))

for mod in modalities:
    print("\nMost similar for id {} and modality {}".format(idx, mod))
    topk_similar = get_topk(evaluator, idx, mod=mod, k=5)
    for sent in topk_similar:
        print("prediction: \t", sent)
       


Most similar for id 178989 and modality text-text
Input sentence:  I don't think I made out with her like at the club.
prediction: 	 I don't think I made out with her like at the club.
prediction: 	 They've been more than like amicable.
prediction: 	 I don't even know how like, I'm an adult
prediction: 	 Definitely my favorite couple.
prediction: 	 So yeah, like half a kiss

Most similar for id 178989 and modality text-audio
Input sentence:  I don't think I made out with her like at the club.
prediction: 	 I don't think I made out with her like at the club.
prediction: 	 They've been more than like amicable.
prediction: 	 No way a couple.
prediction: 	 Give me a kiss.
prediction: 	 push Guinness down the toilet.

Most similar for id 178989 and modality audio-text
Input sentence:  I don't think I made out with her like at the club.
prediction: 	 I don't think I made out with her like at the club.
prediction: 	 No way a couple.
prediction: 	 They've been more than like amicable.
predict

In [257]:
idx = 470482
for mod in modalities:
    print("\nMost similar for id {} and modality {}".format(idx, mod))
    topk_similar = get_topk(evaluator, idx, mod=mod, k=5)
    for sent in topk_similar:
        print("prediction: \t", sent)


Most similar for id 470482 and modality text-text
Input sentence:  Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.

Most similar for id 470482 and modality text-audio
Input sentence:  Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.

Most similar for id 470482 and modality audio-text
Input sentence:  Anchor will distribute your podcast for you.
prediction: 	 GE will distribute your podcast for you.
prediction: 	 